In [1]:
! pip list | grep "torch"

'grep'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [1]:
import re
import sys
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext import data
from torchtext import datasets

## Reading Data

In [2]:

# Data Setting
TEXT = data.Field(batch_first = True,
                  fix_length = 500,
                  tokenize=str.split,
                  pad_first=True,
                  pad_token='[PAD]',
                  unk_token='[UNK]')

LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(text_field = TEXT, 
                                             label_field = LABEL)

In [3]:
# Data Length
print(f'Train Data Length : {len(train_data.examples)}')
print(f'Test Data Length : {len(test_data.examples)}')

Train Data Length : 25000
Test Data Length : 25000


In [4]:
# Data Fields
train_data.fields

{'text': <torchtext.data.field.Field at 0x27a245fc5c0>,
 'label': <torchtext.data.field.LabelField at 0x27a245fc588>}

In [5]:
# Data Sample
print('---- Data Sample ----')
print('Input : ')
print(' '.join(vars(train_data.examples[1])['text']),'\n')
print('Label : ')
print(vars(train_data.examples[1])['label'])

---- Data Sample ----
Input : 
Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the str

## Pre-processing Data

In [6]:
def PreProcessingText(input_sentence):
    input_sentence = input_sentence.lower() # 소문자화
    input_sentence = re.sub('<[^>]*>', repl= ' ', string = input_sentence) # "<br />" 처리
    input_sentence = re.sub('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]', repl= ' ', string = input_sentence) # 특수문자 처리 ("'" 제외)
    input_sentence = re.sub('\s+', repl= ' ', string = input_sentence) # 연속된 띄어쓰기 처리
    if input_sentence:
        return input_sentence

In [7]:
for example in train_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()
    
for example in test_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()

## Making Vocab & Setting Embedding

In [8]:
model_config = {'emb_type' : '', 'emb_dim' : 300}

In [9]:
# no pre-trained 
TEXT.build_vocab(train_data,
                 min_freq = 2, 
                 max_size = None,
                 )

LABEL.build_vocab(train_data)

model_config['vocab_size'] = len(TEXT.vocab)

In [10]:
# Vocabulary Info
print(f'Vocab Size : {len(TEXT.vocab)}')

print('Vocab Examples : ')
for idx, (k, v) in enumerate(TEXT.vocab.stoi.items()):
    if idx >= 10:
        break    
    print('\t', k, v)

print('---------------------------------')

# Label Info
print(f'Label Size : {len(LABEL.vocab)}')

print('Lable Examples : ')
for idx, (k, v) in enumerate(LABEL.vocab.stoi.items()):
    print('\t', k, v)

Vocab Size : 51956
Vocab Examples : 
	 [UNK] 0
	 [PAD] 1
	 the 2
	 and 3
	 a 4
	 of 5
	 to 6
	 is 7
	 in 8
	 it 9
---------------------------------
Label Size : 2
Lable Examples : 
	 neg 0
	 pos 1


## Spliting Validation Data & Making Data Iterator

In [11]:
# Spliting Valid set
train_data, valid_data = train_data.split(random_state = random.seed(0),
                                          split_ratio=0.8)

In [12]:
model_config['batch_size'] = 30

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(datasets=(train_data, valid_data, test_data), 
                                                                           batch_size=model_config['batch_size'],
                                                                           device=device)

# This part

In [13]:
sample_for_check = next(iter(train_iterator))

In [14]:
# Check batch data
sample_for_check = next(iter(train_iterator))
print(sample_for_check)
print(sample_for_check.text)
print(sample_for_check.label)


[torchtext.data.batch.Batch of size 30]
	[.text]:[torch.cuda.LongTensor of size 30x500 (GPU 0)]
	[.label]:[torch.cuda.FloatTensor of size 30 (GPU 0)]
tensor([[   1,    1,    1,  ...,    2,  812,   96],
        [   1,    1,    1,  ..., 1200,    5,   12],
        [   1,    1,    1,  ...,   53,   98, 2187],
        ...,
        [   1,    1,    1,  ...,   10,  368,   78],
        [   1,    1,    1,  ...,   19,    6,   64],
        [   1,    1,    1,  ..., 2571,    5,   99]], device='cuda:0')
tensor([1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.], device='cuda:0')


In [15]:
# Check reverting data
print(' '.join([TEXT.vocab.itos[int(x)] for x in sample_for_check.text[0,:] if x not in [0,1]]))
print(LABEL.vocab.itos[int(sample_for_check.label[0])]) 

ivan marco ricca and alexandre borges are partners in a company together with estevão george but the first ones contract a professional killer anisio to murder estevão the plot at least at the beginning doesn't explain very well why the guy does it and after receiving his money he starts blackmailing the two partners appearing in their company and saying he wants a job there as supervisor or something at the same time he meets marina mariana daughter of estevão and starts dating with her in a story like this where crime corruption betrayal and blackmail go hand in hand no one is innocent or can be victimized exception made to marina which is the only person who doesn't know what's going on and didn't betray anyone this film portrays with sarcasm the sad and cruel reality which exists in big metropolis like são paulo where crime is every day's presence we can feel irony but also veracity in characters like anisio brilliantly played by paulo miklos which does blackmail to the guys who pa

## Making Model

In [16]:
class SentenceClassification(nn.Module):
    def __init__(self, **model_config):
        super(SentenceClassification, self).__init__()

        if model_config['emb_type'] == 'glove' or 'fasttext':
            self.emb = nn.Embedding(num_embeddings = model_config['vocab_size'],
                                    embedding_dim = model_config['emb_dim'],
                                    _weight = TEXT.vocab.vectors)
        else:
            self.emb = nn.Embedding(num_embeddings = model_config['vocab_size'],
                                    embedding_dim = model_config['emb_dim'])
        
        self.bidirectional = model_config['bidirectional']
        self.num_direction = 2 if model_config['bidirectional'] else 1
        self.model_type = model_config['model_type'] 

        self.RNN = nn.RNN (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        
        self.LSTM= nn.LSTM(input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        
        self.GRU = nn.GRU (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
    
        self.fc = nn.Linear(model_config['hidden_dim'] * self.num_direction,
                            model_config['output_dim'])
        
        self.drop = nn.Dropout(model_config['dropout'])

    def forward(self, x):
        
        emb = self.emb(x) 
        # emb : (Batch_Size, Max_Seq_Length, Emb_dim)

        if self.model_type == 'RNN':
            output, hidden = self.RNN(emb) 
        elif self.model_type == 'LSTM':
            output, (hidden, cell) = self.LSTM(emb)
        elif self.model_type == 'GRU':
            output, hidden = self.GRU(emb)
        else:
            raise NameError('Select model_type in [RNN, LSTM, GRU]')
        
        # output : (Batch_Size, Max_Seq_Length, Hidden_dim * num_direction) 
        # hidden : (num_direction, Batch_Size, Hidden_dim)
        # hidden의 경우, batch_first 옵션이 안먹는 문제가 있음
        
        last_output = output[:,-1,:]

        # last_output : (Batch_Size, Hidden_dim * num_direction)
        return self.fc(self.drop(last_output))

### Checking feed-forward

In [17]:
model_config.update(dict(batch_first = True,
                         model_type = 'RNN',
                         bidirectional = True,
                         hidden_dim = 128,
                         output_dim = 1,
                         dropout = 0))

In [18]:
model = SentenceClassification(**model_config).to(device)

In [19]:
predictions = model.forward(sample_for_check.text).squeeze()

In [20]:
loss_fn = nn.BCEWithLogitsLoss().to(device)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum()/len(correct)
    return acc

In [21]:
loss = loss_fn(predictions, sample_for_check.label)
acc = binary_accuracy(predictions, sample_for_check.label)

In [22]:
print(predictions)
print(loss, acc)

tensor([-0.0978,  0.3092, -0.2011, -0.6801,  0.4178,  0.2044,  0.1650,  0.2200,
         0.4264,  0.2468,  0.3407,  0.0169, -0.3391,  0.0502, -0.2879, -0.1104,
        -0.2063, -0.0021,  0.5160,  0.6616,  0.6315, -0.4962, -0.0985, -0.3059,
        -0.3182, -0.1759, -0.3697,  0.4454, -0.3348,  0.5056], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
tensor(0.7777, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>) tensor(0.3333, device='cuda:0')


## Training

In [23]:
def train(model, iterator, optimizer, loss_fn, idx_epoch, **model_params):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() 
    batch_size = model_params['batch_size']

    for idx, batch in enumerate(iterator):
        
        # Initializing
        optimizer.zero_grad()
        
        # Forward 
        predictions = model(batch.text).squeeze()
        loss = loss_fn(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        sys.stdout.write(
                    "\r" + f"[Train] Epoch : {idx_epoch:^3}"\
                    f"[{(idx + 1) * batch_size} / {len(iterator) * batch_size} ({100. * (idx + 1) / len(iterator) :.4}%)]"\
                    f"  Loss: {loss.item():.4}"\
                    f"  Acc : {acc.item():.4}"\
                    )

        # Backward 
        loss.backward()
        optimizer.step()
        
        # Update Epoch Performance
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss/len(iterator) , epoch_acc/len(iterator) 

In [24]:
def evaluate(model, iterator, loss_fn):
    
    epoch_loss = 0
    epoch_acc = 0
    
    # evaluation mode
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = loss_fn(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### bi-RNN

In [25]:
model_config['model_type'] = 'RNN'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [26]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-RNN_
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.532  Acc : 0.73337
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.6426 | Train Acc : 0.6255
	 Epoch : 0 | Valid Loss : 0.6249 | Valid Acc : 0.6546
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.591  Acc : 0.77667
	 Saved at 1-epoch
	 Epoch : 1 | Train Loss : 0.6131 | Train Acc : 0.653
	 Epoch : 1 | Valid Loss : 0.6069 | Valid Acc : 0.6637
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.458  Acc : 0.88333
	 Saved at 2-epoch
	 Epoch : 2 | Train Loss : 0.5001 | Train Acc : 0.7583
	 Epoch : 2 | Valid Loss : 0.5864 | Valid Acc : 0.6983
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.3013  Acc : 0.8667
	 Epoch : 3 | Train Loss : 0.4101 | Train Acc : 0.8204
	 Epoch : 3 | Valid Loss : 0.6013 | Valid Acc : 0.713
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.4681  Acc : 0.7333
	 Epoch : 4 | Train Loss : 0.3619 | Train Acc : 0.8455
	

In [27]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.585 | Test Acc : 0.7046


### bi-LSTM

In [28]:
model_config['model_type'] = 'LSTM'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [29]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-LSTM_
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.388  Acc : 0.86673
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.5992 | Train Acc : 0.6719
	 Epoch : 0 | Valid Loss : 0.4954 | Valid Acc : 0.7604
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.3265  Acc : 0.8667
	 Saved at 1-epoch
	 Epoch : 1 | Train Loss : 0.4223 | Train Acc : 0.8127
	 Epoch : 1 | Valid Loss : 0.4332 | Valid Acc : 0.8202
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.4906  Acc : 0.7667
	 Saved at 2-epoch
	 Epoch : 2 | Train Loss : 0.2878 | Train Acc : 0.8832
	 Epoch : 2 | Valid Loss : 0.4002 | Valid Acc : 0.8396
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.1404  Acc : 0.93337
	 Epoch : 3 | Train Loss : 0.171 | Train Acc : 0.935
	 Epoch : 3 | Valid Loss : 0.418 | Valid Acc : 0.8508
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.2688  Acc : 0.93337
	 Epoch : 4 | Train Loss : 0.09108 | Train Acc : 0.969

In [30]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.4053 | Test Acc : 0.8325


### bi-GRU

In [31]:
model_config['model_type'] = 'GRU'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [32]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-GRU_
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.4023  Acc : 0.9667
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.5662 | Train Acc : 0.7022
	 Epoch : 0 | Valid Loss : 0.4814 | Valid Acc : 0.7778
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.4055  Acc : 0.8333
	 Saved at 1-epoch
	 Epoch : 1 | Train Loss : 0.4068 | Train Acc : 0.8222
	 Epoch : 1 | Valid Loss : 0.4136 | Valid Acc : 0.829
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.2221  Acc : 0.8667
	 Saved at 2-epoch
	 Epoch : 2 | Train Loss : 0.3153 | Train Acc : 0.8643
	 Epoch : 2 | Valid Loss : 0.3651 | Valid Acc : 0.8562
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.3287  Acc : 0.90673
	 Saved at 3-epoch
	 Epoch : 3 | Train Loss : 0.1717 | Train Acc : 0.9361
	 Epoch : 3 | Valid Loss : 0.3541 | Valid Acc : 0.8615
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.05094  Acc : 1.0667
	 Epoch : 4 | Train Loss : 0.07654

In [33]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.3809 | Test Acc : 0.8504


## Inference

In [34]:
model_config['model_type'] = 'GRU'
model = SentenceClassification(**model_config).to(device)
model.load_state_dict(torch.load(f"./{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}.pt"))

<All keys matched successfully>

In [35]:
def predict_sentiment(model, sentence):
    model.eval()
    indexed = TEXT.numericalize(TEXT.pad([TEXT.tokenize(PreProcessingText(sentence))]))
    input_data = torch.LongTensor(indexed).to(device)
    prediction = torch.sigmoid(model(input_data))
    return prediction.item()

In [36]:
test_sentence = 'this movie is FUN'
predict_sentiment(model = model, sentence = test_sentence)

0.8596924543380737